In [2]:
import requests
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize

In [3]:
api_token='9bb4facb6d23f48efbf424bb05c0c1ef1cf6f468393bc745d42179ac4aca5fee'

In [38]:
def process_request(url):
    """
    This function processes the api request and returns a pandas dataframe object
    """
    request = requests.get(url)
    data = json.loads(request.text)
    df = pd.DataFrame(json_normalize(data['result']))
    return df 

In [51]:
def get_threat(name):
    """
    This function gets 
    """
    threat_url = 'http://apiv3.iucnredlist.org/api/v3/threats/species/name/{}?token={}'.format(name,api_token)
    threat_df = process_request(threat_url)
    return threat_df

In [40]:
def get_species_by_group(group_name):
    species_url = 'http://apiv3.iucnredlist.org/api/v3/comp-group/getspecies/{}?token={}'.format(group_name,api_token)
    group_df = process_request(species_url)
    return group_df

In [48]:
def get_species_by_country(country_name):
    country_url = 'http://apiv3.iucnredlist.org/api/v3/country/getspecies/{}?token={}'.format(country_name,api_token)
    country_df = process_request(country_url)
    return country_df

In [41]:
get_threat("Loxodonta africana").head()

,code,invasive,scope,score,severity,timing,title
0,1.1,None,None,Low Impact: 3,None,Ongoing,Housing & urban areas
1,11.2,None,None,Low Impact: 3,None,Ongoing,Droughts
2,1.2,None,None,Low Impact: 3,None,Ongoing,Commercial & industrial areas
3,2.1,None,None,Low Impact: 3,None,Ongoing,Annual & perennial non-timber crops
4,2.1.1,None,None,Low Impact: 3,None,Ongoing,Shifting agriculture


In [42]:
get_species_by_group("seabreams_porgies_picarels").head()

,category,rank,scientific_name,subpopulation,subspecies,taxonid
0,DD,None,Acanthopagrus akazakii,None,None,170230
1,LC,None,Acanthopagrus arabicus,None,None,47166659
2,LC,None,Acanthopagrus australis,None,None,170257
3,LC,None,Acanthopagrus berda,None,None,170266
4,LC,None,Acanthopagrus bifasciatus,None,None,170239


0                           Abies nordmanniana
1       Abies nordmanniana subsp. nordmanniana
2                        Ablepharus bivittatus
3                                Abramis brama
4                             Acanthis flammea
5                      Acanthobrama microlepis
6                             Accipiter badius
7                           Accipiter brevipes
8                           Accipiter gentilis
9                              Accipiter nisus
10                              Acer campestre
11                           Acer cappadocicum
12                               Acer hyrcanum
13                         Acer monspessulanum
14                            Acer platanoides
15                         Acer pseudoplatanus
16                              Acer velutinum
17                     Achnatherum roshevitzii
18                   Acipenser gueldenstaedtii
19                       Acipenser nudiventris
20                          Acipenser persicus
21           

In [52]:
import pycountry

In [66]:
# Obtain a list of ISO2 country codes 
list_iso2 = [list(pycountry.countries)[i].alpha_2 for i in range (len(pycountry.countries))]

In [88]:
def make_master_df():
    # Get a list of country codes using pycountry package. 
    result_df = pd.DataFrame()
    for index,iso2 in enumerate(list_iso2):
        temp_df = get_species_by_country(iso2)
        result_df = result_df.append(temp_df)
        print("---processing {}th country".format(index))
    # We don't want duplicates 
    result_df.drop_duplicates(["scientific_name"], inplace= True)
    return result_df

In [89]:
make_master_df()

---processing 0th country
---processing 1th country
---processing 2th country
---processing 3th country
---processing 4th country
---processing 5th country
---processing 6th country
---processing 7th country
---processing 8th country
---processing 9th country
---processing 10th country
---processing 11th country
---processing 12th country
---processing 13th country
---processing 14th country
---processing 15th country
---processing 16th country
---processing 17th country
---processing 18th country
---processing 19th country
---processing 20th country
---processing 21th country
---processing 22th country
---processing 23th country
---processing 24th country
---processing 25th country
---processing 26th country
---processing 27th country
---processing 28th country
---processing 29th country
---processing 30th country
---processing 31th country
---processing 32th country
---processing 33th country
---processing 34th country
---processing 35th country
---processing 36th country
---processi

,category,rank,scientific_name,subpopulation,subspecies,taxonid
0,LC,None,Ablennes hians,None,None,13486514
1,LC,None,Abudefduf saxatilis,None,None,188581
2,LC,None,Abudefduf taurus,None,None,188434
3,LC,None,Abyssobrotula galatheae,None,None,60469175
4,LC,None,Acanthemblemaria medusa,None,None,47141697
5,LC,None,Acanthocybium solandri,None,None,170331
6,LC,None,Acanthostracion polygonius,None,None,193646
7,LC,None,Acanthurus chirurgus,None,None,177982
8,LC,None,Acanthurus coeruleus,None,None,177953
9,LC,None,Acanthurus tractus,None,None,47139706


In [99]:
# Adding a new column to the dataframe on species class
# Using http://etetoolkit.org/ taxonomy-ranks 0.0.7 lib here to obtain this information
# http://etetoolkit.org/docs/2.3/tutorial/tutorial_ncbitaxonomy.html
from ete3 import NCBITaxa

In [ ]:
ncbi = NCBITaxa()
compiled_species_df = pd.read_csv("datasets/compiled_species.csv")

In [ ]:
compiled_species_df['kingdom'] = compiled_species_df.apply(lambda row: row.a + row.b, axis=1)

In [97]:
dir(tax.name)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'capitalize',
 'casefold',
 'center',
 'count',
 'encode',
 'endswith',
 'expandtabs',
 'find',
 'format',
 'format_map',
 'index',
 'isalnum',
 'isalpha',
 'isascii',
 'isdecimal',
 'isdigit',
 'isidentifier',
 'islower',
 'isnumeric',
 'isprintable',
 'isspace',
 'istitle',
 'isupper',
 'join',
 'ljust',
 'lower',
 'lstrip',
 'maketrans',
 'partition',
 'replace',
 'rfind',
 'rindex',
 'rjust',
 'rpartition',
 'rsplit',
 'rstrip',
 'split',
 'splitlines',
 'startswith',
 'strip',
 'swapcase',
 'title',
 'translate',
 'upper',


In [ ]:
# The dataframe should be  species|threat a
#                          species|threat b
# Otherwise there will be way too many columns
# Only consider high impact threats?? Otherwise there will be a million entries. 
# To speed up the process might use parallel programming in python 
# Ref: https://www.machinelearningplus.com/python/parallel-processing-python/

def check_google_scholar():
    